# NextGEOSS
## Invoke the NOA's Multi-hazard Enhanced Risk Assessment service using JupyterLab

### Inspect the OGC WPS capabilities

In [1]:
from owslib.wps import WebProcessingService
from owslib.wps import monitorExecution
from owslib.wps import printInputOutput

In [2]:
wps_url = "http://stamps-deploy.noa.nextgeoss.terradue.com/wps/WebProcessingService"

In [3]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

In [4]:
wps.getcapabilities()

In [5]:
for operation in wps.operations:
    print operation.name

GetCapabilities
DescribeProcess
Execute


In [6]:
for process in wps.processes:
    print process.identifier

com.terradue.wps_oozie.process.OozieAbstractAlgorithm


### Inspect the OGC WPS process information and inputs

In [7]:
process_id = 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm'
process = wps.describeprocess(process_id)

In [8]:
process.title

'Run Stamps v4.1b'

In [9]:
process.abstract

'Run Stamps v4.1b persistent scatterer with Terradue Hadoop'

In [10]:
for input in process.dataInputs:
    printInputOutput(input)
    print '***'

 identifier=outputdir, title=Processing folder, abstract=Path to Processing directory, data type=string
 Any value allowed
 Default Value: /shared/ath_sent1_test3/stamps_steps_1 
 minOccurs=1, maxOccurs=1
***
 identifier=realrun15, title=Run Steps 1-5, abstract=Values: Yes No, data type=string
 Any value allowed
 Default Value: no 
 minOccurs=1, maxOccurs=1
***
 identifier=runsteps15, title=Run Steps Range (1-5), abstract=Values: x-y (min x = 1, max y = 5), data type=string
 Any value allowed
 Default Value: 1-5 
 minOccurs=1, maxOccurs=1
***
 identifier=realrun57, title=Run Steps 5-7, abstract=Values: Yes No, data type=string
 Any value allowed
 Default Value: no 
 minOccurs=1, maxOccurs=1
***
 identifier=runsteps57, title=Run Steps Range (5-7), abstract=Values: x-y (min x = 5, max y = 7), data type=string
 Any value allowed
 Default Value: 5-7 
 minOccurs=1, maxOccurs=1
***
 identifier=cleanup, title=Clean up processing folder, abstract=Values: Yes No, data type=string
 Any value all

### Prepare the execution request

In [11]:
inputs = []
inputs.append(('outputdir','/shared/ath_sent1_test3/stamps_steps_1'))
inputs.append(('realrun15','no'))
inputs.append(('runsteps15','1-5'))
inputs.append(('realrun57','no'))
inputs.append(('runsteps57','5-7'))
inputs.append(('cleanup','no'))
inputs.append(('pub','no'))


outputs = [('result_osd',True)]

### Execute and monitor the request

In [12]:
execution = wps.execute(process_id, inputs, outputs)

In [13]:
monitorExecution(execution)

In [14]:
execution.isSucceded()

True

### Access the results' OpenSearch Description (osd)

In [15]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [16]:
results_osd = execution.processOutputs[0].reference

In [17]:
print results_osd

http://stamps-deploy.noa.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=4ca86c38-d7f9-4420-be3e-5e3eccb42ea0result_osd.eee13040-cca8-469c-906c-4c42a13600e4


In [18]:
import requests
r = requests.get(results_osd)
r.text

u'<?xml version=\'1.0\' encoding=\'UTF-8\'?><ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" href="http://stamps-deploy.noa.nextgeoss.terradue.com/sbws/wps/dcs-stamps4-aa/0000003-190621125042955-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>'

In [19]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
root.attrib['href']

'http://stamps-deploy.noa.nextgeoss.terradue.com/sbws/wps/dcs-stamps4-aa/0000003-190621125042955-oozie-oozi-W/results/description'

### Perform an OpenSearch query for the results enclosures

In [20]:
import cioppy

In [21]:
ciop = cioppy.Cioppy()

In [22]:
search = ciop.search(end_point = root.attrib['href'],
                     params = [],
                     output_fields='enclosure')

In [23]:
for result in search:
    print result['enclosure']

### Download an output file

The following section can be leveraged when published results are available.

In [24]:
#search[0]['enclosure']

In [25]:
#r = requests.get(search[0]['enclosure'],stream=True)

In [26]:
#with open('output', 'wb') as fd:
#    for chunk in r.iter_content(chunk_size=128):
#        fd.write(chunk)